# 이미지의 바운딩 박스를 gui로 쉽게 그리자 아하

## 바운딩할 꽃 종류를 정하자

In [1]:
import os
import cv2
import numpy as np
import shutil

In [3]:
flower_label = {
                'buquet' : 0,
                'rose' : 0,             # 장미
                'empty1' : 1,
                'tulip' : 2,            # 튤립
                'carnation' : 3,
                'empty2' : 4,        # 카네이션
                'lily' : 5,             # 백합
                'lisianthus' : 6,       # 리시안셔스
                'small_chrysanthemum' : 7,    # 소국
                'pompom_blooms' : 8,    # 퐁퐁국화
                'sunflower': 9,         # 해바라기
                'gerbera' : 10,          # 거베라
                'alstroemeria' : 11,     # 알스트로메리아
                'hydrangea' : 12,       # 수국
                'peony' : 13,           # 작약
                'stock' : 14,           # 스톡
                'freesia' : 15,         # 프리지아
                'ranunculus' : 16,      # 라넌큘러스
                'empty3' : 17,
                'butterfly':18,  # 라넌큘러스 버터플라이
                'cala': 19,               # 칼라 꽃
                'snapdragon' : 20,        # 금어초
                }

flower_name_list = [i[0] for i in sorted([[k,v] for k,v in flower_label.items()], key=lambda x: x[1])]

flower = 'freesia'

f_num = flower_label[flower]

path = f'./0_flower_data/{flower}'

In [4]:
# 마우스 이벤트 콜백 함수
def mouse_callback(event, x, y, flags, param):
    global point1, point2, cnt, img, coordi, f_label_name, f_num
    if event == cv2.EVENT_LBUTTONDOWN:
        if cnt%2:
            img_copy = img.copy()
            point1 = (x, y)
            cv2.line(img_copy, (x, y), (x, img.shape[0]), (0, 255, 0), 5)
            cv2.line(img_copy, (x, y), (img.shape[1], y), (0, 255, 0), 5)
            update_text(img_copy, f_label_name)
            
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
        else:
            px, py = point1
            point2 = (x, y)
            
            cv2.line(img, (px, py), (px, y), (0, 255, 0), 5)
            cv2.line(img, (px, py), (x, py), (0, 255, 0), 5)
            cv2.line(img, (x, y), (px, y), (0, 255, 0), 5)
            cv2.line(img, (x, y), (x, py), (0, 255, 0), 5)
            
            img_copy = img.copy()
            update_text(img_copy, f_label_name)
            
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
            
            
            if point1 and point2:
                # 좌측상단 좌표와 우측하단 좌표로 변환
                x1, y1 = min(point1[0], point2[0]), min(point1[1], point2[1])
                x2, y2 = max(point1[0], point2[0]), max(point1[1], point2[1])
                
                # yolo의 바운딩박스 수치로 변환
                x = round((x1+(x2-x1)/2)/img.shape[1], 4)
                y = round((y1+(y2-y1)/2)/img.shape[0], 4)
                w = round((x2-x1)/img.shape[1], 4)
                h = round((y2-y1)/img.shape[0], 4)
                
                coordi += f"{f_num} {x} {y} {w} {h}\n"
                # 좌표 저장
            
        cnt += 1

def update_text(image, f_label_name):
    font = cv2.FONT_HERSHEY_SIMPLEX
    x, y = image.shape[:2]
    font_scale = (x+y)//1000
    font_color = (255, 125, 60)
    line_type = (x+y)//600
    position = [x//10,y//10]
    text = f'{f_label_name} : {flower_label[f_label_name]}'
    cv2.putText(image, text, position, font, font_scale, font_color, line_type)

break_plag = False
flower_list = os.listdir(path)

if not os.path.isdir(f'./1_labels_data/{flower}'):
    os.mkdir(f'./1_labels_data/{flower}')

f_idx = 0

while 1:

    f_name = flower_list[f_idx]
    f_num = flower_label[flower]
    
    f_name_idx = flower_name_list.index(flower)
    f_label_name = flower
    
    if break_plag:
        break
    
    # 이미지 파일 경로
    img_path = f'./0_flower_data/{flower}/{f_name}'

    # 이미지 로드 및 윈도우 생성
    img = cv2.imread(img_path)
    
    if type(img) != np.ndarray:
        os.remove(f'./0_flower_data/{flower}/{f_name}')
        continue
    
    height, width = img.shape[:2]
    
    if height > width:
        new_height = 640
        new_width = int(width * (new_height / height))
    else:
        new_width = 640
        new_height = int(height * (new_width / width))
    
    cv2.namedWindow(f'{f_name} {width} * {height}', cv2.WINDOW_NORMAL)
    cv2.resizeWindow(f'{f_name} {width} * {height}', (new_width, new_height))
    cv2.setMouseCallback(f'{f_name} {width} * {height}', mouse_callback)
    
    img_copy = img.copy()
    update_text(img_copy, f_label_name)
    
    cv2.imshow(f'{f_name} {width} * {height}', img_copy)

    # 마우스 이벤트를 통한 좌표 저장
    point1, point2, coordi, cnt = None, None, '', 1
     
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        # 좌표를 txt 파일로 저장하고 다음으로 넘어가기
        # 만약 좌표를 그리지 않았다면 저장하지 않고 그냥 넘어가기
        if key == ord("q"):
            f_idx += 1
            if coordi == '':
                break
            
            with open(f"./1_labels_data/{flower}/{f_name[:-4]}.txt", "a") as f:
                f.write(coordi)
            break
        
        # 다음 꽃 바운딩
        elif key == ord('e'):
            cnt = 1

            f_name_idx = f_name_idx+1 if len(flower_name_list)-1 > f_name_idx else 0
            f_label_name = flower_name_list[f_name_idx]
            f_num = flower_label.get(f_label_name)

            img_copy = img.copy()
            update_text(img_copy, f_label_name)
            
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
            
        # 이전 꽃 바운딩
        elif key == ord('w'):
            cnt = 1

            f_name_idx = f_name_idx-1 if 0 < f_name_idx else len(flower_name_list)-1
            f_label_name = flower_name_list[f_name_idx]
            f_num = flower_label.get(f_label_name)
            
            img_copy = img.copy()
            update_text(img_copy, f_label_name)
            
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
            
        # # 다음 세부 클래스 바운딩
        # elif key == ord('d'):
        #     cnt = 1
        #     form_len = len(flower_label.get(flower_name_list[f_name_idx]))
        #     f_form = f_form+1 if f_form < form_len-1 else 0

        #     img_copy = img.copy()
        #     update_text(img_copy, f_label_name, f_form)
            
        #     cv2.imshow(f'{f_name} {width} * {height}', img_copy)
        
        # 저장하지 않은 좌표들을 지우고, 다시 그리기
        elif key == ord('c'):
            cnt = 1
            img = cv2.imread(img_path)
            img_copy = img.copy()
            update_text(img_copy, f_label_name)
            coordi = ''
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
            
        elif key == ord('m'):
            cnt = 1
            with open(f"./1_labels_data/{flower}/{f_name[:-4]}.txt", "w") as f:
                f.write('')
        
        # 볼 것도 없다. 데이터셋에서 그냥 삭제
        elif key == ord('f'):
            os.remove(f'./0_flower_data/{flower}/{f_name}')
            flower_list = os.listdir(f'./0_flower_data/{flower}')
            break

        # 지나쳐버렸다... 앞으로 돌아가기
        elif key == ord('z'):
            if f_idx > 0:
                f_idx -= 1
                break
            elif f_idx == 0:
                pass
        
        # 여기서 끝내기!
        elif key == ord('p'):
            break_plag = True
            break

    # 윈도우 종료 및 리소스 해제q
    cv2.destroyAllWindows()

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: './0_flower_data/freesia'

## 바운딩 박스 그리기가 끝난 애들을 2로 넘기자

In [16]:
import shutil

bounding_txt = {i[:-4]:True for i in os.listdir(f'./1_labels_data/{flower}')}
img_list = [i[:-4] for i in os.listdir(f'./0_flower_data/{flower}')]

if not os.path.isdir(f'./2_learning_img/{flower}'):
    os.mkdir(f'./2_learning_img/{flower}')

for i in img_list:
    if bounding_txt.get(i):
        shutil.copy(f'./0_flower_data/{flower}/{i}.jpg', f'./2_learning_img/{flower}/{i}.jpg')

### 3의 이전 파일들 삭제하기

In [17]:
for C in ['train', 'test']:
    for L in ['labels', 'images']:
        for i in os.listdir(f'./3_one_time_data/{C}/{L}'):
            os.remove(f'./3_one_time_data/{C}/{L}/{i}')
    
    if os.path.isfile(f'./3_one_time_data/{C}/labels.cache'):
        os.remove(f'./3_one_time_data/{C}/labels.cache')

## 2의 이미지와 1의 라벨들을 3으로 나눠 보내자

In [18]:
import random

bounding_txt = os.listdir(f'./1_labels_data/{flower}')
img_list = os.listdir(f'./2_learning_img/{flower}')

random_list = random.sample([i for i in range(len(img_list))], len(img_list)//10*2)

for i in range(len(img_list)):
    CLS = 'train'
    img = img_list[i]
    label = img[:-4]+'.txt'
    
    if i in random_list:
        CLS = 'test'
    
    shutil.move(f'./2_learning_img/{flower}/{img}', f'./3_one_time_data/{CLS}/images/{img}', )
    shutil.move(f'./1_labels_data/{flower}/{label}', f'./3_one_time_data/{CLS}/labels/{label}')

# 1차 학습 시작

python train.py --img 640 --rect --batch 16 --epochs 100 --weight yolov5m.pt --cfg one_yolov5m.yaml --data one_data.yaml --name buquet

# 이 아래는 파일 관리

### 3의 파일을 돌리는 코드

In [3]:
for C in ['train', 'test']:
    for L in ['labels', 'images']:
        for i in os.listdir(f'./3_one_time_data/{C}/{L}'):
            if L == 'images':
                shutil.move(f'./3_one_time_data/{C}/{L}/{i}', f'./2_learning_img/{flower}/{i}')
            else:
                shutil.move(f'./3_one_time_data/{C}/{L}/{i}', f'./1_labels_data/{flower}/{i}')
    
    if os.path.isfile(f'./3_one_time_data/{C}/labels.cache'):
        os.remove(f'./3_one_time_data/{C}/labels.cache')

### 자리 밀렸을 때, 인덱스 미루기

In [7]:
for C in ['train', 'test']:
    for L in os.listdir(f'./5_total_data/{C}/labels/'):
        if L[0:2] =='al':
            with open(f'./5_total_data/{C}/labels/{L}', 'r') as f:
                x = ''
                for i in f.readlines():
                    x += '11' + i[2:]
            
            with open(f'./5_total_data/{C}/labels/{L}', 'w') as f:
                f.write(x)

### 5의 특정 파일을 3으로 복사

In [ ]:
F = ''

for CLS in ['train', 'test']:
    
    for i in os.listdir(f'./5_total_data/{CLS}/images')